# Explore relationships to build a normal model

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def get_connection(schema="proj001_lfb_0_0_1"):
    
    conn = psycopg2.connect("dbname='proj001_lfb' user='postgres' "
                            "host='10.152.234.53'")
    if schema:
        cur = conn.cursor()
        query = "SET search_path TO {}".format(schema)
        cur.execute(query)
        
    return conn

# Incident address

## Eastings and Northings
These should be unique within a postcode

In [6]:
conn = get_connection()
qry = ('select distinct easting_m, northing_m, count(distinct postcode_full) '
          'from proj001_lfb_0_0_1.l2_incidents '
          'group by  easting_m, northing_m '
          'having count(distinct postcode_full) >1 '
          'order by count(distinct postcode_full) desc ;')
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
data = pd.read_sql(qry, conn)  

In [7]:
data.head(10)

,easting_m,northing_m,count
0,529625,180625,49
1,529125,180375,41
2,529125,180625,38
3,528875,180875,36
4,529125,181375,35
5,529125,181125,32
6,533750,181250,31
7,529625,180875,30
8,531625,181125,30
9,528875,181125,29


Inspect the 2 worst cases to see what is going on

In [11]:
qry = ("select incidentnumber_cln, postcode_full, easting_m, northing_m "
           "from proj001_lfb_0_0_1.l2_incidents "
            "where (easting_m = '529625' and northing_m = '180625') "
            "or (easting_m = '529125' and northing_m = '180375') ;")
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
data = pd.read_sql(qry, conn)  
data.head(20)

,incidentnumber_cln,postcode_full,easting_m,northing_m
0,3627091,W1S 4NG,529125,180375
1,8124091,W1B 5RR,529625,180625
2,10139091,SW1Y 6HB,529625,180625
3,10456091,W1S 4NN,529125,180375
4,12055091,W1S 4HA,529125,180375
5,12924091,W1J 0DU,529125,180375
6,14343091,W1D 6BA,529625,180625
7,17240091,W1S 4HP,529125,180375
8,17935091,W1J 8DL,529125,180375
9,19103091,W1S 4NG,529125,180375


# Here's a particularly bad inconsistency

In [15]:
qry = ("select * "
           "from proj001_lfb_0_0_1.l2_incidents "
           "where incidentnumber_cln in ('29342091','31291091', '17935091' , '19103091') " )
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
data = pd.read_sql(qry, conn)  
data

,incidentnumber_cln,dateofcall_cln,incidentgroup,stopcodedescription,specialservicetype,propertycategory,propertytype,addressqualifier,postcode_full,incgeo_boroughname,incgeo_wardname,incgeo_wardnamenew,easting_m,northing_m,frs,incidentstationground,numstationswithpumpsattending,numpumpsattending
0,17935091,2009-01-31 15:00:28+00:00,False Alarm,False alarm - Good intent,None,Non Residential,Restaurant/cafe,Correct incident location,W1J 8DL,WESTMINSTER,ST. JAMES'S,ST. JAMES'S,529125,180375,LONDON,SOHO,3,3
1,19103091,2009-02-02 10:54:24+00:00,False Alarm,AFA,None,Non Residential,Purpose built office,Correct incident location,W1S 4NG,WESTMINSTER,ST. JAMES'S,ST. JAMES'S,529125,180375,LONDON,SOHO,1,1
2,29342091,2009-02-20 03:14:25+00:00,False Alarm,AFA,None,Non Residential,Purpose built office,Within same building,SW1Y 4SP,WESTMINSTER,ST. JAMES'S,ST. JAMES'S,529625,180625,LONDON,SOHO,1,1
3,31291091,2009-02-23 09:27:45+00:00,Fire,Primary Fire,None,Non Residential,Single shop,Within same building,W1B 4EG,WESTMINSTER,ST. JAMES'S,ST. JAMES'S,529625,180625,LONDON,SOHO,2,3


So while Eastings and Northings seem inaccurate within postcode, these postcodes seem to be adjacent. Is this just small location errors in the Eastings and Northings?

## How many easting/northing combinations within a postcode?

In [22]:
qry = ("select distinct postcode_full,  easting_m, northing_m, count(*) "
           "from proj001_lfb_0_0_1.l2_incidents "
           "group by postcode_full,  easting_m, northing_m "
           "order by postcode_full, count(*) desc ;" )

data = pd.read_sql(qry, conn)  
data.head(20)

,postcode_full,easting_m,northing_m,count
0,AL1 3EA,514850,207450,1
1,AL2 1QY,518215,203627,1
2,BR1 1AE,540194,169201,4
3,BR1 1AE,540250,169250,1
4,BR1 1BQ,540299,169607,1
5,BR1 1BW,540381,168612,1
6,BR1 1BY,539955,169594,1
7,BR1 1DD,540364,169136,6
8,BR1 1DD,540350,169133,1
9,BR1 1DE,539934,169644,1


So we will model post code as one entity and then easting/northing as the specific incident address

## Property
Look into the fields that should be part of a property entity.

In [4]:
conn = get_connection()
qry = ("select distinct propertycategory, propertytype, addressqualifier "
           "from proj001_lfb_0_0_1.l2_incidents "
           "order by propertycategory, propertytype, addressqualifier;" )

data = pd.read_sql(qry, conn)  
data.head(20)

,propertycategory,propertytype,addressqualifier
0,Aircraft,Freight plane,In street remote from gazetteer location
1,Aircraft,Freight plane,Nearby address - no building in street
2,Aircraft,Freight plane,Nearby address - street not listed in gazetteer
3,Aircraft,Freight plane,On land associated with building
4,Aircraft,Helicopter,Correct incident location
5,Aircraft,Helicopter,On land associated with building
6,Aircraft,Light aircraft,Correct incident location
7,Aircraft,Light aircraft,Nearby address - street not listed in gazetteer
8,Aircraft,Light aircraft,On land associated with building
9,Aircraft,Military helicopter,Open land/water - nearest gazetteer location


It seems like `addressqualifier` is less important

In [5]:
conn = get_connection()
qry = ("select distinct  addressqualifier "
           "from proj001_lfb_0_0_1.l2_incidents "
           "order by addressqualifier;" )

data = pd.read_sql(qry, conn)  
data

,addressqualifier
0,Correct incident location
1,In street close to gazetteer location
2,In street outside gazetteer location
3,In street remote from gazetteer location
4,Nearby address - no building in street
5,Nearby address - street not listed in gazetteer
6,On land associated with building
7,On motorway / elevated road
8,Open land/water - nearest gazetteer location
9,Railway land or rolling stock
